#### Task 1: Simple vector embedding generation

**Objective:**
Generate vector embeddings from text data.

**Task Description:**

- load huggingface embedding model (``model_name="sentence-transformers/all-mpnet-base-v2"``)
- embed simple text queries

How to select the right embedding model: [MTEB - Massive Text Embedding Benchmark](https://huggingface.co/blog/mteb)

**Useful links:**

- [Langchain Chroma](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/)


In [25]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document

# ADD HERE YOUR CODE
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [26]:
text = "This is a test document."
# ADD HERE YOUR CODE
# Perform vector search
query_vector = embedding_model.embed_query(text)

print(f"Embedding vector length: {len(query_vector)}")
print(query_vector[:10])

Embedding vector length: 768
[-0.04895174875855446, -0.039861924946308136, -0.021562796086072922, 0.009908466599881649, -0.03810402378439903, 0.012684384360909462, 0.04349454864859581, 0.07183399051427841, 0.009748565033078194, -0.006986980792135]


#### Task 2: Generate embedding vectors with custom dataset

**Objective:**
Load custom dataset, preprocess it and generate vector embeddings.

**Task Description:**

- load pdf document "AI_Book.pdf" via langchain document loader: ``PyPDFLoader``
- use RecursiveCharacterTextSplitter to split documents into chunks
- generate embeddings for single documents

**RecursiveCharacterTextSplitter:**

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough. The default list is `["\n\n", "\n", " ", ""]`. This has the effect of trying to keep all paragraphs (and then sentences, and then words) together as long as possible, as those would generically seem to be the strongest semantically related pieces of text.

**Useful links:**

- [Langchain PyPDFLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.pdf.PyPDFLoader.html)
- [Langchain RecursiveCharacterTextSplitter](https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html)


In [27]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import re

pdf_doc = "./AI_Book.pdf"

# Create pdf data loader
# ADD HERE YOUR CODE
loader = PyPDFLoader(file_path=pdf_doc)

# Load and split documents in chunks
# ADD HERE YOUR CODE
pages = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size = 10000, chunk_overlap = 20)
pages_chunked = splitter.split_documents(pages)

# Function to clean text by removing invalid unicode characters, including surrogate pairs
def clean_text(text):
    # Remove surrogate pairs
    text = re.sub(r'[\ud800-\udfff]', '', text)
    # Optionally remove non-ASCII characters (depends on your use case)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    return text

def clean_and_create_document(chunk):
    cleaned_text = clean_text(chunk.page_content)
    return Document(page_content=cleaned_text, metadata=chunk.metadata)


pages_chunked_cleaned = [clean_and_create_document(chunk) for chunk in pages_chunked]


print(pages_chunked[0])

page_content='Aurélien GéronHands-on Machine Learning with
Scikit-Learn, Keras, and
TensorFlow
Concepts, Tools, and Techniques to
Build Intelligent SystemsSECOND EDITION
Boston Farnham Sebastopol Tokyo Beijing Boston Farnham Sebastopol Tokyo Beijing' metadata={'source': './AI_Book.pdf', 'page': 2}


In [28]:
print(pages_chunked[1])

page_content='978-1-492-03264-9
[LSI]Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow
by Aurélien Géron
Copyright © 2019 Aurélien Géron. All rights reserved.
Printed in the United States of America.
Published by O’Reilly Media, Inc., 1005 Gravenstein Highway North, Sebastopol, CA 95472.
O’Reilly books may be purchased for educational, business, or sales promotional use. Online editions are
also available for most titles ( http://oreilly.com ). For more information, contact our corporate/institutional
sales department: 800-998-9938 or corporate@oreilly.com .
Editor:  Nicole Tache
Interior Designer:  David FutatoCover Designer:  Karen Montgomery
Illustrator:  Rebecca Demarest
June 2019:  Second Edition
Revision History for the Early Release
2018-11-05: First Release
2019-01-24: Second Release
2019-03-07: Third Release
2019-03-29: Fourth Release
2019-04-22: Fifth Release
See http://oreilly.com/catalog/errata.csp?isbn=9781492032649  for release details.
The O’Reilly logo 

In [29]:
print(f"Number of text chunks: {len(pages_chunked)}")

Number of text chunks: 501


#### Task 3: Store vector embeddings from pdf document to ChromaDB vector database.

**Objective:**

Store vector embeddings into ChromaDB to store knowledge.

**Task Description:**

- create chromadb client
- create chromadb collection
- create langchain chroma db client
- store text document chunks and vector embeddings to vector databases

**Useful links:**

- [Langchain How To](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/#initialization-from-client)

In [30]:
from langchain_chroma import Chroma
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

client = chromadb.HttpClient(
    host="localhost",
    port=8000,
    ssl=False,
    headers=None,
    settings=Settings(allow_reset=True, anonymized_telemetry=False),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)

# ADD HERE YOUR CODE
# Create a collection
collection = client.get_or_create_collection("collection_name")

# ADD HERE YOUR CODE
# Create chromadb
vector_db_from_client = Chroma(
    client=client,
    collection_name="collection_name",
    embedding_function=embedding_model,
    )

In [31]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(pages_chunked_cleaned[:50]))]

vector_db_from_client.add_documents(documents=pages_chunked_cleaned[:50], id = uuids)


#uuids = [str(uuid4()) for _ in range(len(pages_chunked_cleaned[:50]))]

# ADD HERE YOUR CODE
#vector_db_from_client.add_texts(pages_chunked_cleaned, id = uuids)

['5aa84e75-f1ec-4dd5-941d-4646cb096cab',
 '45ba4feb-4386-4af8-af66-8a311dd5042d',
 '3745340f-be4e-4534-ac60-e65072acc5dc',
 '7812062e-b398-4fd9-adfb-8ee974b7c069',
 'bd86a47f-cb6f-4062-a5a1-39dd1385e209',
 '5bef9f2e-1d96-42ce-a247-561666f61177',
 'e44617cb-6120-444d-ae93-faaa371472b3',
 'ea4faf90-d35f-40af-8ca2-b998202726df',
 'e5a6e95e-2487-415d-a836-1b73d260c954',
 'ea7037f3-06cc-4630-97e2-03458275a0ac',
 '854841de-c761-42de-bcc6-69e4be35883f',
 'ffd1f756-a0d0-4323-aa11-db5b73b0d76e',
 '1957788c-d675-4496-bc50-73085d7b3a89',
 'b3bc4545-5ea5-47f9-ad46-939469e908cb',
 '9f19928d-9108-4af0-b41a-c98f931df2f7',
 'eafe1fce-8c25-4e4a-8052-adcb37319676',
 '4ab9ae81-71df-48ac-b65d-6e1e1d88be75',
 'f5af1b80-3055-465e-90a2-87fd675b40a1',
 '754a8354-98b8-43ae-93ff-be733d4f9d05',
 '6edb633d-acac-4854-9d31-573e2edf0791',
 '1ec71f64-eaa2-4f43-9dfc-e8ca5e64ddcd',
 'c9b53637-594c-46fa-b954-2277814538a3',
 'a41dbca2-e69b-4dfb-994f-4c5a76a5357f',
 'f87b63af-0271-4585-8e25-37cdec1b63ac',
 'fbe04ae6-eb28-

In [32]:
client.count_collections()

1

In [33]:
# client.delete_collection("ai_model_book")

#### Task 4: Access ChromaDB and perform vector search

**Objective:**

Use query to perform vector search against ChromaDB vector database

**Task Description:**

- define query
- run vector search
- print k=3 most similar documents


**Useful links:**

- [Langchain Query ChromaDB](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/#query-directly)

In [34]:
search_query = str("Types of Machine Learning Systems")

results = vector_db_from_client.similarity_search(
    "search_query",
    k=3,
    )

for res in results:
    print(res.page_content)
    print(res.metadata)
    print("\n")

Frame the Problem                                                                                                    39
Select a Performance Measure                                                                                  42
Check the Assumptions                                                                                             45
Get the Data                                                                                                                    45
Create the Workspace                                                                                                45
Download the Data                                                                                                    49
Take a Quick Look at the Data Structure                                                                50
Create a Test Set                                                                                                          54
Discover and Visualize the Data to Gain Insights